<a href="https://colab.research.google.com/github/tahkhan-lab/tahmina/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contractions

import pandas as pd
import string

import re
import contractions
import nltk


In [ ]:
df = pd.read_csv('messages.csv')
df = df.iloc[:1000]
def clean_ling_spam_text(text):
    text = contractions.fix(text)                          # Expand contractions
    text = text.lower()                                    # Convert to lowercase
    text = re.sub(r'http\S+|www\S+', '', text)             # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)                    # Remove email addresses
    text = re.sub(r'\d+', '', text)                        # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)                    # Remove punctuation
    text = re.sub(r'\s+', ' ', text)                       # Normalize spaces
    return text.strip()                                    # Strip leading/trailing spaces

# Apply to dataset
df['text'] = df['subject'].fillna('') + ' ' + df['message'].fillna('')
df['clean_text'] = df['text'].apply(clean_ling_spam_text)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(df['label'])
X_train, X_test, y_train, y_test = train_test_split( df['clean_text'], Y,  test_size=0.2, random_state=42)


In [ ]:
!pip install tensorflow-text==2.18.0
!pip install tensorflow==2.18.0 --upgrade


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub


preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)


def get_bert_embeddings(texts):
    text_input = tf.constant(texts)
    preprocessed_text = preprocessor(text_input)
    outputs = encoder(preprocessed_text)
    pooled_output = outputs['pooled_output']
    return pooled_output


In [ ]:
train_embeddings = get_bert_embeddings(X_train.tolist())
test_embeddings = get_bert_embeddings(X_test.tolist())

print(train_embeddings.shape)
print(type(train_embeddings))
print(test_embeddings[0])

(800, 768)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[-8.28500092e-01 -5.41028321e-01 -8.60333502e-01  6.95926249e-01
  6.13716543e-01 -2.66683906e-01  4.78107274e-01  5.46989620e-01
 -6.51389360e-01 -9.99972880e-01 -2.79742450e-01  8.54702950e-01
  9.76797342e-01  3.51474524e-01  7.02075303e-01 -6.29951298e-01
  6.77920952e-02 -6.89379036e-01  4.91997898e-01  4.71280158e-01
  5.08619130e-01  9.99993443e-01 -2.33929157e-01  5.17639458e-01
  5.41282415e-01  8.93631339e-01 -7.21429527e-01  8.65937352e-01
  8.77734125e-01  6.07307434e-01 -3.84033293e-01  5.71109116e-01
 -9.80852902e-01 -3.29713076e-01 -9.07189250e-01 -9.87330437e-01
  5.55304766e-01 -5.61674953e-01 -8.37630704e-02 -1.44289061e-01
 -7.10846066e-01  5.74426055e-01  9.99969304e-01 -1.06053106e-01
  5.30742109e-01 -4.32799965e-01 -9.99999523e-01  4.05368835e-01
 -6.62435949e-01  8.23661149e-01  5.52685380e-01  8.67721438e-01
  2.59018868e-01  5.60411036e-01  4.54443306e-01 -3.13509494e-01
  9.46815982e-

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
rf_model = RandomForestClassifier()
rf_model.fit(train_embeddings, y_train)

# Predict on the test set
y_pred = rf_model.predict(test_embeddings)

print (accuracy_score(y_test,y_pred))
print (confusion_matrix(y_test,y_pred))

0.96
[[165   1]
 [  7  27]]


Comparison
In conclusion, Our implementation of BERT embeddings with a Random Forest classifier achieved a strong accuracy of 96% on the Ling-Spam dataset. The referenced paper, which also evaluates Random Forest as part of its baseline comparisons, reports a significantly higher 99.14% accuracy using a hybrid GWO-BERT model that integrates BERT, LSTM, and the Grey Wolf Optimizer for optimized feature selection. While our approach demonstrates the effectiveness of combining BERT with classical machine learning, the paper shows that performance can be further improved by incorporating deep learning and optimization techniques.
